[View in Colaboratory](https://colab.research.google.com/github/VAgni/Demo/blob/master/catsvsdogs.ipynb)

In [0]:
import numpy as np
import os
import tensorflow as tf
from random import shuffle
from tqdm import tqdm
import cv2


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

TRAIN_DIR='/content/gdrive/My Drive/train/'
IMG_SIZE= 50
LR= 1e-3
MODEL_NAME= 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic-video')

NOW WE START DATA PREPROCESSING OF IMAGES

In [0]:
def label_img(img):
  word_label=img.split('.')[0]
  if word_label=='cat':return[1,0]
  elif word_label=='dog': return[0,1]

In [0]:
def create_train_data():
  training_data = [] #made an empty array to store the images
  for img in tqdm(os.listdir(TRAIN_DIR)): #loop to get all the images from trainig set
    label=label_img(img)
    path= os.path.join(TRAIN_DIR,img) #giving path from directory to imag
    img= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img= cv2.resize(img, (IMG_SIZE,IMG_SIZE))#reading image from directory and converting it into grayscale 
    training_data.append([np.array(img), np.array(label)])#appending the label to its corresponding image
  shuffle(training_data)# to randomize the data
  np.save('train_data.npy', training_data)
  return training_data
    
    

In [0]:
def create_test_data():
  testing_data = []
  for img in tqdm(os.listdir(TEST_DIR)):
    path =  os.path.join(TEST_DIR,img)
    img_num= img.split('.')[0]
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
  testing_data.append([np.array(img), np.array(img_num)])
  np.save('test_data.npy', testing_data)
  return testing_data
  

In [6]:
train_data= create_train_data()

100%|██████████| 4287/4287 [23:55<00:00,  3.34it/s]


In [7]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 2.4MB/s 
  Running setup.py bdist_wheel for tflearn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [8]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [0]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
  model.load(MODEL_NAME)
  print('model loaded!')



In [0]:
train = train_data[:-500]
test = train_data[-500:]
X = np.array([i[0] for i in  train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]
test_x=  np.array([i[0] for i in  test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y= [i[1] for i in test]



In [12]:
model.fit({'input': X}, {'targets': Y}, n_epoch=10, 
          validation_set=({'input': test_x}, {'targets': test_y}), 
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1199  | total loss: 5.53477 | time: 19.491s
| Adam | epoch: 020 | loss: 5.53477 - acc: 0.7596 -- iter: 3776/3787
Training Step: 1200  | total loss: 5.44901 | time: 20.822s
| Adam | epoch: 020 | loss: 5.44901 - acc: 0.7634 | val_loss: 4.69727 - val_acc: 0.7960 -- iter: 3787/3787
--


In [13]:
model.save(MODEL_NAME)

INFO:tensorflow:/content/dogsvscats-0.001-2conv-basic-video.model is not in all_model_checkpoint_paths. Manually adding it.
